In [6]:
import networkx as nx
import pickle
import json
import torch

G = nx.read_gpickle("../data/graph_cleaned.pickle")  # defs from wordsapi

In [7]:
type(G)

networkx.classes.graph.Graph

In [8]:
G['active']

AtlasView({'passive': {'weight': -1}, 'involved': {'weight': 1}, 'inactive': {'weight': -1}, 'engaged in or ready for military or naval operations': {'relation': 'defined as'}, "(used of verbs (e.g. `to run') and participial adjectives (e.g. `running' in `running water')) expressing action rather than a state of being": {'relation': 'defined as'}, 'taking part in an activity': {'relation': 'defined as'}, 'in operation': {'relation': 'defined as'}, 'characterized by energetic activity': {'relation': 'defined as'}, 'disposed to take action or effectuate change': {'relation': 'defined as'}, 'engaged in full-time work': {'relation': 'defined as'}, 'exerting influence or producing a change or effect': {'relation': 'defined as'}, 'expressing that the subject of the sentence has the semantic function of actor:': {'relation': 'defined as'}, 'full of activity or engaged in continuous activity': {'relation': 'defined as'}, '(of e.g. volcanos) capable of erupting': {'relation': 'defined as'}, '(o

### Integrate sentence similarity scoring.

In [9]:
from transformers import AutoModel, AutoTokenizer, BertTokenizer
MODEL_NAME = "bert-base-cased"

# We need to create the model and tokenizer
model = AutoModel.from_pretrained(MODEL_NAME)
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

In [10]:
pooled_list = []  # list will contain a tuple <trait, definition, bert representation of the definition>.
for node1, node2, edge in G.edges(data='relation'):
    if edge == 'defined as':
        tokens = tokenizer.tokenize(node2)
        tokens_ids = tokenizer.convert_tokens_to_ids(tokens)

        tokens_ids = tokenizer.build_inputs_with_special_tokens(tokens_ids)


        tokens_pt = torch.tensor([tokens_ids])
        _, pooled_temp = model(tokens_pt)
        pooled_temp = pooled_temp.squeeze(0)
        pooled_list.append((node1, node2, pooled_temp))

In [11]:
def tokenize_sentence(sentence):
    tokens = tokenizer.tokenize(sentence)
    #print("Tokens: {}".format(tokens))

    # This is not sufficient for the model, as it requires integers as input, 
    # not a problem, let's convert tokens to ids.
    tokens_ids = tokenizer.convert_tokens_to_ids(tokens)
    #print("Tokens id: {}".format(tokens_ids))

    # Add the required special tokens
    tokens_ids = tokenizer.build_inputs_with_special_tokens(tokens_ids)

    # We need to convert to a Deep Learning framework specific format, let's use PyTorch for now.
    tokens_pt = torch.tensor([tokens_ids])
    #print("Tokens PyTorch: {}".format(tokens_pt))

    # Now we're ready to go through BERT with out input
    outputs, pooled = model(tokens_pt)
    #print("Token wise output: {}, Pooled output: {}".format(outputs.shape, pooled.shape))
    pooled = pooled.squeeze(0)
    return pooled

In [12]:
def find_similar_trait(s, pooled_list):
    """
    :param s: a candidate sentence.
    :param pooled_list: a list of tuples <trait, definition, tensor_rep_of_sentence>.
    """
    pooled = tokenize_sentence(s)
    maxi = -1
    mini = 1
    count_thres = 0
    THRES = 0.99
    for trait, defn, rep in pooled_list:
        similarity = torch.cosine_similarity(rep, pooled, 0).item()
        if similarity > maxi:
            maxi = similarity
            max_defn = defn
            max_trait = trait
        if similarity >= THRES:
            count_thres = count_thres + 1
#         if similarity < mini:
#             mini = similarity
#             min_defn = defn
#             min_trait = trait
    return {
        'max': maxi,
        'max_defn': max_defn,
        'max_trait': max_trait,
        'count_thres': count_thres
    }

In [35]:
def find_traits(s1, s2, pooled_list):
    """
    :param s1: a candidate sentence.
    :param s2: another candidate sentence.
    :param pooled_list: a list of tuples <trait, definition, tensor_rep_of_sentence>.
    """

    result = find_similar_trait(s1, pooled_list)
    #display(result['max'], result['max_defn'], result['max_trait'], result['count_thres'])
    trait1 = result['max_trait']

    result = find_similar_trait(s2, pooled_list)
    #display(result['max'], result['max_defn'], result['max_trait'], result['count_thres'])
    trait2 = result['max_trait']
    
    return trait1, trait2

In [15]:
import pandas as pd
with open("../data/roberta/astro0.tsv") as f:
    dataset = pd.read_csv(f, delimiter='\t')

In [16]:
dataset.head()

,Unnamed: 0,judge,first,second
0,0,entailment,"People with Virgo rising tend to be practical,...","You have an ingenious, active and alert mind."
1,1,entailment,"People with Virgo rising tend to be practical,...",Gaining knowledge and putting it to good use a...
2,2,entailment,"People with Virgo rising tend to be practical,...",You strive for perfection and can be quite the...
3,3,entailment,"People with Virgo rising tend to be practical,...","At times others can never be ""good"" enough to ..."
4,4,entailment,"People with Virgo rising tend to be practical,...",Finding fault with what's wrong with things is...


### Graph inference of trait relations.

In [18]:
import torch
from fairseq.data.data_utils import collate_tokens
from math import exp
import numpy as np

class RobertaMNLI:
    # todo: create similar wrapper classes for other NLI engines.
    # todo: create a wrapper superclass and subclass from there.
    
    def __init__(self, rel_path):
        self.output_map = {
            0: 'contradiction',
            1: 'neutral',
            2: 'entailment'
        }
        
        torch.hub.set_dir(rel_path)
        self.roberta = torch.hub.load('pytorch/fairseq', 'roberta.large.mnli')  # works
        self.roberta.eval()
        
    def predict_one(self, S1, S2, return_probs=False):
        batch = collate_tokens(
            [self.roberta.encode(S1, S2)], pad_idx=1
        )
        logprobs = self.roberta.predict('mnli', batch)
        classes_tsr = logprobs.argmax(dim=1)
        classes = classes_tsr.tolist()  
        
        if return_probs == True:
            logprobs_list=[logprob.item() for logprob in logprobs[0]]
            prob_list =  [pow(exp(1), logprob) for logprob in logprobs_list]
            return prob_list
        else:
            return classes[0]  # 0 is contradiction, 1 is neutral, 2 is entailment.

In [19]:
predictor = RobertaMNLI(rel_path="../../roberta/hub")

Using cache found in ../../roberta/hub\pytorch_fairseq_master


In [20]:
probs = predictor.predict_one('auspicious', 'bright', return_probs=True)
cls = predictor.output_map[np.argmax(probs)]
print('predicted class: ', cls)
print('probabilities: ', probs)

predicted class:  entailment
probabilities:  [0.2838515728606713, 0.3048106316015907, 0.41133781278178166]


In [21]:
def entailment_score(predictor, S1, S2):
    """
    Returns entailment score of S1 and S2.
    We ignore the directionality of S1 and S2; we return the higher entailment score
    regardless of whether S1 entails S2 or S2 entails S1.
    
    :param predictor: wrapper class for NLI engine. See RobertaMNLI above.
    :param S1: String.
    :param S2: String.
    """
    probs1 = predictor.predict_one(S1, S2, return_probs=True)
    probs2 = predictor.predict_one(S2, S1, return_probs=True)
    e1 = probs1[2]  # index 2 is for entailment.
    e2 = probs2[2]
    return max(e1, e2)
    
    
def infer_sentence_relation(trait_dict, trait_graph, s1, s2):
    """
    :param trait_dict: Dict of trait definitions; the traits in this dict are a subset of the
    traits in trait_graph.
    :param trait_graph: Graph of antonym/synonym relations between traits.
    :param s1: String. for comparison w s2.
    :param s2: String. for comparison w s1.
    """
    # Find likely trait1 and trait2.
    trait1_scores = dict()
    trait2_scores = dict()
    for trait, defs in trait_dict:
        trait1_scores[trait] = list()
        trait2_scores[trait] = list()
        for definition in defs:
            score = entailment_score(definition, s1)
            trait1_scores[trait].append(score)
            
            score = entailment_score(definition, s2)
            trait2_scores[trait].append(score)
            
    # Obtain highest scoring trait1 and trait2 from trait scores dicts.
    # {traitx: list_of_scores}
    
    # Given trait1 and trait2, find edges between them and infer if antonym or synonym.


In [22]:
s1 = "you are a very brave man."
s2 = "you are a courageous person."
entailment_score(predictor, s1, s2)

0.9935886904587607

In [38]:
def get_paths_of_weights(G, paths_list):
    """
    Return the weights of the shortest paths.
    
    :param G: source graph
    :param paths_list: all shortest paths. e.g. networkx.all_shortest_paths().
    """
    weight_paths_list = list()
    for path in paths_list:
        weight_path = list()
        for i in range(len(path) - 1):
            edge_attr = G.get_edge_data(path[i], path[i+1])
            weight_path.append(edge_attr['weight'])
        weight_paths_list.append(weight_path)
    return weight_paths_list

def get_entailment_paths_of_weights(predictor, paths_list):
    """
    Return the entailment scores along the shortest paths.
    
    :param predictor: wrapper class for NLI engine. See RobertaMNLI above.
    :param paths_list: all shortest paths. e.g. networkx.all_shortest_paths().
    """
    weight_paths_list = list()
    for path in paths_list:
        weight_path = list()
        for i in range(len(path) - 1):
            score = entailment_score(predictor, path[i], path[i+1])
            weight_path.append(score)
        weight_paths_list.append(weight_path)
    return weight_paths_list

# todo: need to write the algorithm that accounts for antonyms along the inference path.
'''
if 1 get entailment score, if -1 get contradiction score. this will help us see what is the strength
of the synonym/antonym relations along the inference path.
however, still unsure of how to account for decaying word sense.
'''

def vis_weighted_paths(paths_list, weight_paths_list):
    # visualise the weighted_paths
    for i in range(len(paths_list)):
        path = paths_list[i]
        weights = weight_paths_list[i]
        to_print = str()
        for j in range(len(path)-1):
            to_print = to_print + str(f"{path[j]}-({weights[j]:.3f})-")
        to_print = to_print + f"{path[-1]}"
        print(to_print + '\n')
        
def infer_trait_relations(paths_of_weights):
    """
    simple algorithm for computing relation between start and end node of path.
    :param paths_of_weights: see get_paths_of_weights(G, paths_list) above. 
    weights are strictly 1 or -1.
    """
    inferred_relations = list()
    for weight_path in paths_of_weights:
        start_val = weight_path[0]
        for i in weight_path[1:]:
            start_val = start_val * i
        inferred_relations.append(start_val)
    return inferred_relations
       
inference_map = {-1: "contradiction", 1: "entailment"}  # todo: make this a Class attribute.

def choose_rel(inferred_relations):
    """
    from a list of graph-inferred relations, chooses the mode as the final inferred relation.
    """
    try:
        return inference_map[mode(inferred_relations)]
    except StatisticsError as e:
        # no mode found. synonym and antonym equally likely
        return "neutral"
        
def print_rel_counts(inferred_relations):
    distribution = np.unique(inferred_relations, return_counts=True)
    print("<relation: count>:")
    for i in range (len(distribution[0])):
        print(f"{inference_map[distribution[0][i]]}: {distribution[1][i]}")

In [39]:
import random
import numpy as np
from statistics import mode
from statistics import StatisticsError

G2 = nx.read_gpickle("../data/traits_only_graph_cleaned2.pickle")

In [41]:
inferences = list()
for i in range(len(dataset)):
    s1 = dataset.iloc[i]['first']
    s2 = dataset.iloc[i]['second']
    display(s1, s2)
    t1, t2 = find_traits(s1, s2, pooled_list)

    #t1 = random.choice(list(G2.nodes))  # example trait
    #t1 = 'ungrudging'
    print(t1)
    #t2 = random.choice(list(G2.nodes))  # example trait
    #t2 = 'disagreeable'
    print(t2)

    if t1 != t2:
        try:
            paths = nx.all_shortest_paths(G2, source=t1, target=t2, weight=None, method='dijkstra')
            paths_list = [row for row in paths]

            paths_of_weights = get_paths_of_weights(G2, paths_list)
            #vis_weighted_paths(paths_list, paths_of_weights)

            #paths_of_scores = get_entailment_paths_of_weights(predictor, paths_list)
            #vis_weighted_paths(paths_list, paths_of_scores)

            inferred_relations = infer_trait_relations(paths_of_weights)

            rel = choose_rel(inferred_relations)


        except nx.NetworkXNoPath as e:
            rel = "neutral"
    else:
        rel = "entailment"

    inferences.append(rel)

'People with Virgo rising tend to be practical, analytical, discriminating, fastidious, careful, exacting, attentive to details, methodical, quiet, unassuming, shy, critical, thoughtful, and somewhat self-centered.'

'You have an ingenious, active and alert mind.'

thick
thick


'People with Virgo rising tend to be practical, analytical, discriminating, fastidious, careful, exacting, attentive to details, methodical, quiet, unassuming, shy, critical, thoughtful, and somewhat self-centered.'

'Gaining knowledge and putting it to good use are important to you.'

...

'You aim to please in practical and earthy ways.'

thoughtless
disagreeable


'Guard against possessiveness, jealousy, and taking the easy way out in your relationships.'

'You aim to please in practical and earthy ways.'

ambitious
disagreeable


In [42]:
inferences

['entailment',
 'entailment',
 'entailment',
 'neutral',
 'contradiction',
 'contradiction',
 'neutral',
 'entailment',
 'neutral',
 'contradiction',
 'entailment',
 'neutral',
 'entailment',
 'neutral',
 'neutral',
 'contradiction',
 'entailment',
 'entailment',
 'entailment',
 'entailment',
 'entailment',
 'contradiction',
 'contradiction',
 'contradiction',
 'neutral',
 'entailment',
 'entailment',
 'entailment',
 'neutral',
 'entailment',
 'neutral',
 'entailment',
 'neutral',
 'entailment',
 'neutral',
 'neutral',
 'entailment',
 'neutral',
 'entailment',
 'entailment',
 'entailment',
 'neutral',
 'entailment',
 'contradiction',
 'neutral',
 'entailment',
 'entailment',
 'contradiction',
 'neutral',
 'entailment',
 'entailment',
 'entailment',
 'contradiction',
 'entailment',
 'entailment',
 'entailment',
 'contradiction',
 'contradiction',
 'entailment',
 'entailment',
 'entailment',
 'neutral',
 'contradiction',
 'contradiction',
 'neutral',
 'entailment',
 'neutral',
 'contradi

In [44]:
graph_inferences = pd.DataFrame(data=inferences, columns=['inference'])
graph_inferences.head()

,inference
0,entailment
1,entailment
2,entailment
3,neutral
4,contradiction


In [45]:
graph_inferences.to_csv("../data/kg_inferences_on_astro0_1.csv")

In [46]:
equals = list()
for i in range(len(graph_inferences)):
    equal = dataset.iloc[i]['judge'] == graph_inferences.iloc[i]['inference']
    equals.append(equal)
np.unique(equals, return_counts=True)

(array([False,  True]), array([986, 784], dtype=int64))

In [47]:
np.unique(graph_inferences, return_counts=True)

(array(['contradiction', 'entailment', 'neutral'], dtype=object),
 array([313, 645, 812], dtype=int64))

### Try to use Wordnet to determine if adjacent words have similar word sense (for tracking word sense decay).

In [48]:
np.unique(dataset['judge'], return_counts=True)

(array(['contradiction', 'entailment', 'neutral'], dtype=object),
 array([  16,  266, 1488], dtype=int64))

In [119]:
import numpy as np
from nltk.corpus import wordnet as wn
from itertools import product

wordx, wordy = "dull","inactive"
sem1, sem2 = wn.synsets(wordx, pos='a'), wn.synsets(wordy, pos='a')  # gives pos == 'a' or 's'.

sem1 = [sem for sem in sem1 if sem.pos() == 'a' ]
sem2 = [sem for sem in sem2 if sem.pos() == 'a' ]

display(list(product(sem1,sem2)))

scores = list()
for i,j in list(product(sem1,sem2)):
    score = i.lch_similarity(j)
    scores.append(score)
print(scores)
 
scores2 = list()
for score in scores:
    if score != None:
        scores2.append(score)
    else:
        scores2.append(np.nan)
max_index = np.nanargmax(scores2)
print(max_index)
print(scores2[max_index])
list(product(sem1,sem2))[max_index]

a


[(Synset('dull.a.01'), Synset('inactive.a.02')),
 (Synset('dull.a.01'), Synset('nonoperational.a.01')),
 (Synset('dull.a.01'), Synset('inactive.a.04')),
 (Synset('dull.a.01'), Synset('dormant.a.02')),
 (Synset('dull.a.01'), Synset('passive.a.01')),
 (Synset('dull.a.01'), Synset('inactive.a.07')),
 (Synset('dull.a.01'), Synset('inactive.a.08')),
 (Synset('dull.a.01'), Synset('inactive.a.09')),
 (Synset('dull.a.02'), Synset('inactive.a.02')),
 (Synset('dull.a.02'), Synset('nonoperational.a.01')),
 (Synset('dull.a.02'), Synset('inactive.a.04')),
 (Synset('dull.a.02'), Synset('dormant.a.02')),
 (Synset('dull.a.02'), Synset('passive.a.01')),
 (Synset('dull.a.02'), Synset('inactive.a.07')),
 (Synset('dull.a.02'), Synset('inactive.a.08')),
 (Synset('dull.a.02'), Synset('inactive.a.09')),
 (Synset('dull.a.06'), Synset('inactive.a.02')),
 (Synset('dull.a.06'), Synset('nonoperational.a.01')),
 (Synset('dull.a.06'), Synset('inactive.a.04')),
 (Synset('dull.a.06'), Synset('dormant.a.02')),
 (Synse

[None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None]


ValueError: All-NaN slice encountered